In [2]:
# BIBLIOTEKI
import pyodbc
import pandas as pd
import numpy as np
import posixpath as os
import gc

#from matplotlib import pyplot as plt
#from matplotlib.pyplot import figure
from IPython.display import display, HTML
from pandas.tseries.offsets import *
from datetime import timedelta

pd.options.display.max_rows=500
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

p=r'C:\Users\DW\KOPALNIA\FINANSE'

In [3]:
import pyodbc
pyodbc.drivers()

['ODBC Driver 17 for SQL Server']

In [4]:
# ZAPYTANIE SQL

t_sql="SELECT RozrachunkiIdx.Typ, RozrachunkiIdx.Dokument, RozrachunkiIdx.DokumentType, RozrachunkiIdx.Podmiot, Kontrahenci.Nazwa, RozrachunkiIdx.PodmiotType, RozrachunkiIdx.Numer, RozrachunkiIdx.Data,  RozrachunkiIdx.KwotaValue, RozrachunkiIdx.Termin, RozrachunkiIdx.DataRozliczenia, RozrachunkiIdx.KwotaRozliczonaValue, RozrachunkiIdx.KwotaRozliczonaSymbol, RozrachunkiIdx.DoRozliczeniaValue FROM            RozrachunkiIdx INNER JOIN Kontrahenci ON RozrachunkiIdx.Podmiot = Kontrahenci.ID WHERE (RozrachunkiIdx.Typ = 10) AND (RozrachunkiIdx.PodmiotType = 'Kontrahenci') AND (RozrachunkiIdx.Termin BETWEEN CONVERT(DATETIME, '2022-01-01 00:00:00', 102) AND CONVERT(DATETIME, '2022-12-31 00:00:00', 102))"


cnxn=pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                    'Server=192.168.1.252\ENOVA_KOPALNIA;'
                    'Database=OGORZELEC;'
                    'UID=ro_daniel.wojcik;'
                    'PWD=661594050xkr@;'
                    'Trusted_Connection=no;')

df=pd.read_sql_query (t_sql,cnxn)

t_nal_raport=df[['Nazwa', 'Numer', 'Data', 'KwotaValue', 'Termin', 'DataRozliczenia','KwotaRozliczonaValue', 'DoRozliczeniaValue']].copy()
t_nal_raport=t_nal_raport[t_nal_raport['DoRozliczeniaValue']>0]

t_nal_raport['VAL_REC_DUE']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if x['Termin']<=(pd.to_datetime('today')) else 0, axis=1)
t_nal_raport['VAL_DUE_99_14']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if x['Termin']<(pd.to_datetime('today')-timedelta(days=14)) else 0, axis=1)

t_nal_raport['VAL_DUE_14_07']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if (x['Termin']<(pd.to_datetime('today')-timedelta(days=7)) and x['Termin']>=(pd.to_datetime('today')-timedelta(days=14))) else 0, axis=1)
t_nal_raport['VAL_DUE_07_00']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if (x['Termin']<(pd.to_datetime('today')-timedelta(days=0)) and x['Termin']>=(pd.to_datetime('today')-timedelta(days=7))) else 0, axis=1)
t_nal_raport['VAL_FUT_00_07']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if (x['Termin']>=(pd.to_datetime('today')+timedelta(days=0)) and x['Termin']<(pd.to_datetime('today')+timedelta(days=7))) else 0, axis=1)
t_nal_raport['VAL_FUT_07_14']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if (x['Termin']>=(pd.to_datetime('today')+timedelta(days=7)) and x['Termin']<(pd.to_datetime('today')+timedelta(days=14))) else 0, axis=1)
t_nal_raport['VAL_FUT_14_99']=t_nal_raport.apply(lambda x:x['DoRozliczeniaValue'] if (x['Termin']>=(pd.to_datetime('today')+timedelta(days=14))) else 0, axis=1)
t_nal_raport.rename(columns={'DoRozliczeniaValue':'VAL_REC_ALL'},inplace=True)


t_nal_raport=t_nal_raport.groupby(['Nazwa'],as_index=False)[['VAL_REC_ALL','VAL_REC_DUE','VAL_DUE_99_14','VAL_DUE_14_07', 'VAL_DUE_07_00', 'VAL_FUT_00_07', 'VAL_FUT_07_14','VAL_FUT_14_99']].sum()
t_nal_raport=t_nal_raport.sort_values(by=['VAL_REC_DUE'],ascending=False)
t_nal_raport.loc['RAZEM']=t_nal_raport.sum(numeric_only=True,axis=0)
t_nal_raport

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Nazwa,VAL_REC_ALL,VAL_REC_DUE,VAL_DUE_99_14,VAL_DUE_14_07,VAL_DUE_07_00,VAL_FUT_00_07,VAL_FUT_07_14,VAL_FUT_14_99
20,KOBYLARNIA S.A.,"1,782,872.75","794,568.49","198,569.28","211,723.59","384,275.62",0.00,"401,403.15","586,901.11"
45,UNIBEP SPÓŁKA AKCYJNA,"532,585.33","532,585.33",0.00,0.00,"532,585.33",0.00,0.00,0.00
10,CEMEX POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIAL...,"1,052,572.05","307,859.16","157,613.43","13,530.00","136,715.73","239,734.38","259,342.50","245,636.01"
36,Przedsiębiorstwo Produkcji Mas Betonowych BOST...,"943,310.90","274,536.00",0.00,0.00,"274,536.00","9,061.48","3,925.03","655,788.39"
15,HDTS POWER s.r.o.,"400,668.09","163,946.66","163,946.66",0.00,0.00,"71,389.24","29,503.56","135,828.63"
18,Jeleniogórskie Przedsiębiorstwo Robót Drogowyc...,"137,967.23","51,953.70","34,313.51","6,540.72","11,099.47","17,087.65","11,959.91","56,965.97"
43,TRANS-PIOTR Transport Ciężarowy Piotr Brener,"41,013.59","27,391.68","4,380.65","7,212.20","15,798.83","13,621.91",0.00,0.00
9,BUDIMEX SA,"52,555.49","15,295.12",0.00,604.37,"14,690.75","6,861.98",0.00,"30,398.39"
34,PRZEDSIĘBIORSTWO PRODUKCYJNO-HANDLOWO -USŁUGOW...,"15,271.64","15,271.64","14,964.09",0.00,307.55,0.00,0.00,0.00
33,PRZEDSIĘBIORSTWO HANDLOWO USŁUGOWE MON-MAR,"14,671.34","14,671.34","14,671.34",0.00,0.00,0.00,0.00,0.00


In [25]:
t_klient=df[['Nazwa', 'PodmiotType','Numer', 'Data', 'Termin', 'KwotaValue', 'DoRozliczeniaValue']].copy()
t_klient=t_klient[(t_klient['Termin']<=pd.to_datetime('today'))&(t_klient['DoRozliczeniaValue']!=0)].sort_values(by='Termin',ascending=True)
# t_klient.loc['RAZEM']=t_klient.sum(numeric_only=True,axis=0)
t_klient_agg=t_klient.groupby(['Nazwa'],as_index=False)[['DoRozliczeniaValue']].sum()
t_klient_agg=t_klient_agg.sort_values(by='DoRozliczeniaValue',ascending=False)
t_klient_agg

,Nazwa,DoRozliczeniaValue
14,KOBYLARNIA S.A.,"794,568.49"
29,UNIBEP SPÓŁKA AKCYJNA,"532,585.33"
6,CEMEX POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIAL...,"307,859.16"
24,Przedsiębiorstwo Produkcji Mas Betonowych BOST...,"274,536.00"
9,HDTS POWER s.r.o.,"163,946.66"
12,Jeleniogórskie Przedsiębiorstwo Robót Drogowyc...,"51,953.70"
27,TRANS-PIOTR Transport Ciężarowy Piotr Brener,"27,391.68"
5,BUDIMEX SA,"15,295.12"
23,PRZEDSIĘBIORSTWO PRODUKCYJNO-HANDLOWO -USŁUGOW...,"15,271.64"
22,PRZEDSIĘBIORSTWO HANDLOWO USŁUGOWE MON-MAR,"14,671.34"


In [26]:
for index, row in t_klient_agg.iterrows():
    n=row['Nazwa']
    t=t_klient[t_klient['Nazwa']==n]
    display(t)

,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2034,KOBYLARNIA S.A.,Kontrahenci,FV/22/08/0071,2022-08-10,2022-09-09,"198,569.28","198,569.28"
2104,KOBYLARNIA S.A.,Kontrahenci,FV/22/08/0110,2022-08-16,2022-09-15,"211,723.59","211,723.59"
2123,KOBYLARNIA S.A.,Kontrahenci,FV/22/08/0147,2022-08-19,2022-09-18,"194,758.20","194,758.20"
2153,KOBYLARNIA S.A.,Kontrahenci,FV/22/08/0178,2022-08-24,2022-09-23,"189,517.42","189,517.42"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2154,UNIBEP SPÓŁKA AKCYJNA,Kontrahenci,FV/22/09/0011,2022-09-02,2022-09-23,"532,585.33","532,585.33"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2033,CEMEX POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIAL...,Kontrahenci,FV/22/08/0065,2022-08-10,2022-09-09,"157,613.43","157,613.43"
2096,CEMEX POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIAL...,Kontrahenci,FVP/22/08/0007,2022-08-31,2022-09-14,"13,530.00","13,530.00"
2145,CEMEX POLSKA SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIAL...,Kontrahenci,FV/22/08/0166,2022-08-23,2022-09-22,"136,715.73","136,715.73"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2144,Przedsiębiorstwo Produkcji Mas Betonowych BOST...,Kontrahenci,FV/22/08/0062,2022-08-08,2022-09-22,"137,268.00","137,268.00"
2143,Przedsiębiorstwo Produkcji Mas Betonowych BOST...,Kontrahenci,FV/22/08/0058,2022-08-08,2022-09-22,"137,268.00","137,268.00"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
1299,HDTS POWER s.r.o.,Kontrahenci,FD/22/05/0017,2022-05-24,2022-06-23,"8,210.47","8,210.47"
1300,HDTS POWER s.r.o.,Kontrahenci,FD/22/05/0018,2022-05-24,2022-06-23,"22,665.59","22,665.59"
1773,HDTS POWER s.r.o.,Kontrahenci,FD/22/07/0007,2022-07-12,2022-08-11,"27,979.04","27,979.04"
1824,HDTS POWER s.r.o.,Kontrahenci,FD/22/07/0009,2022-07-18,2022-08-17,"22,126.86","22,126.86"
1896,HDTS POWER s.r.o.,Kontrahenci,FD/22/07/0015,2022-07-26,2022-08-25,"31,901.07","31,901.07"
1958,HDTS POWER s.r.o.,Kontrahenci,FD/22/08/0003,2022-08-02,2022-09-01,"27,404.11","27,404.11"
2018,HDTS POWER s.r.o.,Kontrahenci,FD/22/08/0006,2022-08-08,2022-09-07,"23,659.52","23,659.52"


...

,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2160,TRANSPORT CIĘŻAROWY NAKONIECZNY LESZEK,Kontrahenci,FV/22/09/0071,2022-09-09,2022-09-23,"1,229.07","1,229.07"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2166,"P.H.U. ""LITBUD"" KRZYSZTOF LITWA",Kontrahenci,FV/22/09/0084,2022-09-09,2022-09-23,"1,036.77","1,036.77"


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
1575,AKMANN JAN RUDY,Kontrahenci,KFV 3/000001/22,2022-07-14,2022-07-21,600.00,600.00


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2066,LAFARGE KRUSZYWA I BETON Sp. z o.o.,Kontrahenci,FVK/22/08/0003,2022-08-30,2022-09-11,541.82,541.82


,Nazwa,PodmiotType,Numer,Data,Termin,KwotaValue,DoRozliczeniaValue
2067,BEST PARTNER Andrzej Czachor,Kontrahenci,FVU/22/09/0001,2022-09-02,2022-09-11,61.50,61.50
